In [1]:
from Narratives import *


In [2]:
narratives_init(
    n_tenant_id = '<your-tenant-id>',
    n_client_id = '<your-client-id>',
    n_client_secret = '<your-clinet-secret>',
    n_sub_key = '<api-key>',
    
    n_deployment_id = "gpt-4o",
    n_api_version = "2024-06-01",
    
    n_workspace_name = "EOU 360 Reporting",
    n_dataset_name = 'Weather Impact Report',
    n_timestamp = datetime.datetime.now()
)

## Get Current year
q = f"""
EVALUATE
VAR curYear =  CALCULATE( MAX('Global_WTI_Weather_New_Model'[Year]),'Global_WTI_Weather_New_Model'[Forecast] = "0")
return ROW("Year",curYear)
"""
current_year = PowerBI_Query(q)["[Year]"][0]

## Get Current Month
q = f"""
EVALUATE
VAR MaxMonth = CALCULATE( MAX('Global_WTI_Weather_New_Model'[MonthNumber]),'Global_WTI_Weather_New_Model'[NSR_YTD_YTG] = "YTD",'dim Dates'[Year] = {current_year})
return SUMMARIZECOLUMNS('Global_WTI_Weather_New_Model'[Month],FILTER('Global_WTI_Weather_New_Model','Global_WTI_Weather_New_Model'[MonthNumber] = MaxMonth))
"""
current_month = PowerBI_Query(q)["Global_WTI_Weather_New_Model[Month]"][0]

Access Token: eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6ImltaTBZMnowZFlLeEJ0dEFxS19UdDVoWUJUayIsImtpZCI6ImltaTBZMnowZFlLeEJ0dEFxS19UdDVoWUJUayJ9.eyJhdWQiOiJodHRwczovL2FuYWx5c2lzLndpbmRvd3MubmV0L3Bvd2VyYmkvYXBpIiwiaXNzIjoiaHR0cHM6Ly9zdHMud2luZG93cy5uZXQvNTQ4ZDI2YWItOGNhYS00OWUxLTk3YzItYTFiMWEwNmNjMzljLyIsImlhdCI6MTczOTU0Nzc0MCwibmJmIjoxNzM5NTQ3NzQwLCJleHAiOjE3Mzk1NTE2NDAsImFpbyI6ImsyUmdZR2dVa092N0pqdVBWU3FxVFAralUwUWlBQT09IiwiYXBwaWQiOiI4OTNkNTA1My0zZDQwLTQ1ZjAtOThmMy02OWY4NzljMGVjY2UiLCJhcHBpZGFjciI6IjEiLCJpZHAiOiJodHRwczovL3N0cy53aW5kb3dzLm5ldC81NDhkMjZhYi04Y2FhLTQ5ZTEtOTdjMi1hMWIxYTA2Y2MzOWMvIiwiaWR0eXAiOiJhcHAiLCJvaWQiOiI2N2MyYTdjYi0wOTEwLTQ1MmQtOTE0Yy0wNzEzMGJlMTFjNTQiLCJyaCI6IjEuQVNZQXF5YU5WS3FNNFVtWHdxR3hvR3pEbkFrQUFBQUFBQUFBd0FBQUFBQUFBQUFtQUFBbUFBLiIsInN1YiI6IjY3YzJhN2NiLTA5MTAtNDUyZC05MTRjLTA3MTMwYmUxMWM1NCIsInRpZCI6IjU0OGQyNmFiLThjYWEtNDllMS05N2MyLWExYjFhMDZjYzM5YyIsInV0aSI6ImY3UXJoVVEtVmtDMWVaYWpmUEYxQUEiLCJ2ZXIiOiIxLjAiLCJ4bXNfaWRyZWwiOiI3IDEyIn0.hrIv9e1mSx7EA-i2l-yYyaGjO

In [3]:
drill_metric = "[Weather Impact mUC]"
prompt = """
Produce a formal narrative of this data. Do not include anything other than the narrative in your response. 
Instructions: 
Data is weather impact on sales. 
Business Unit and Country represent the key. 
There is metrics for Temperature, Precipitation and Cloud Coverage. 
Each show the deviation vs the prior year (PY) as well as an Impact % expected. 
The Weather Impact % is the total weather impact % expected, the sum of the temperature, precipitation and cloud coverage impacts. 
Talk about the key contributors to weather both in the units of measurement of the metric as well as the % impacts. 
Rank the impacts relative to Weather Impact mUC but don't discuss this metric in the narrative. 
Avoid using overly superfluous words (significant, modest, notable) to describe performance. 
Include only the narrative in your response. 
"""
summary_prompt="""
Rewrite this long narrative to build a report out of our performance this month. 
Currently each subsection was written without context of it's parent section. 
I want a new narrative that can look at all of these sections holistically and produce an insightful description of Europe's performance. 
The hierarchy follows Europe to Buyer to Bottler to Country. 
So if you see certain channels or categories are performing at each level of the drill down, note this as a single insight rather than repeating it. 
The narrative should be 'Buyer X is doing well everywhere'. Write me a narrative in the context of reporting out the monthly performance for the company. 
Keep it factual and insightful.
"""
metrics = """
"Temperature CY vs PY (C)", FORMAT( [Temperature CY vs PY], "0.0") & "C",
"Temperature Impact %", FORMAT([Temperature Impact %] * 100, "0.00") & "%",
"Precipitation CY vs PY (mm)", FORMAT([Precipitation CY vs PY], "0.0") & "mm",
"Precipitation Impact %", FORMAT([Precipitation Impact %] * 100, "0.00") & "%",
"Cloud Coverage CY vs PY (%)", FORMAT([Cloud Coverage CY vs PY], "0.0") & "%",
"Cloud Coverage Impact %", FORMAT([Cloud Coverage Impact %] * 100, "0.00") & "%",
"Weather Impact %", FORMAT([Weather Impact %] * 100, "0.00") & "%",
"Weather Impact mUC", FORMAT([Weather Impact mUC]/1000000, "0.0") & "mUC"
"""
filters = f"""
FILTER (
		'dim Countries',
		'dim Countries'[Operating Unit] = "Europe" &&
        NOT ('dim Countries'[Country] IN {{"Russia", "Luxembourg"}})
	),
FILTER(
	'dim Dates',
	'dim Dates'[Year] = {current_year} &&
	'dim Dates'[Month] = "{current_month}"
)
"""

query_json = {
    'filters': filters,
    'metrics':metrics,
    'drill_metric': drill_metric,
    'dimensions': ["'dim Countries'[Operating Unit]","'dim Countries'[Zone]","'dim Countries'[Country]"]
}

In [ ]:
out = iterate_dynamic(query_json, 2)
narrative = iterateNarratives(out,prompt,summary_prompt)
entry={
    "Report":dataset_name,
    "Table":f"{current_year}_{current_month}_narrative",
    "Narrative":narrative,
    "Timestamp":timestamp
}
write_dict_to_excel("Narratives.xlsx", entry)

Grouping: ["'dim Countries'[Operating Unit]"] Ranking Top 2: 'dim Countries'[Zone]
Grouping: ["'dim Countries'[Operating Unit]", "'dim Countries'[Zone]"] Ranking Top 2: 'dim Countries'[Country]


'In reviewing our monthly performance in Europe, we observe varied impacts of weather conditions on sales across different countries and regions, revealing both challenges and opportunities. \n\nIn the broader regional analysis, Europe East and Europe West have experienced distinct weather impacts influencing sales. Europe East was significantly affected by a temperature increase of 0.9°C, driving a sales impact of 0.77%, despite negligible changes in precipitation and cloudiness. In contrast, Europe West saw a reduction in cloud coverage as a major positive influence, contributing 0.56% to sales, whereas temperature and precipitation played more moderate roles, resulting in a total weather impact of 0.59%.\n\nDrilling down further, in specific countries within these regions, diverse weather conditions have played crucial roles in shaping sales outcomes. Remarkably, Italy was negatively impacted, with a notable reduction in cloud coverage accounting for a -1.45% sales impact and an ove